### Método Seq2Seg
Para este caso vamos a implementar un modelo generador de titulos de las noticias, vamos a entrenar con las noticias scrapeadas del diario gestion.

### Instalación de Librerías

In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install pandas nltk tqdm

Looking in indexes: https://download.pytorch.org/whl/cpu


In [2]:
import nltk
nltk.download('stopwords', quiet=True)

True

### Implementación del Modelo Seq2Seq

In [3]:
import os
import re
import random
import json
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import nltk
from nltk.corpus import stopwords

# -------------------------------------------------
# 0. Parámetros principales (ajústalos según tu GPU/CPU)
# -------------------------------------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tamaños de vocabulario máximos (puedes variar)
MAX_VOCAB_SRC = 20000    # Para tokens de "summit"
MAX_VOCAB_TGT = 5000     # Para tokens de "title"

EMBEDDING_DIM = 100
HIDDEN_DIM = 256

# Máxima longitud de secuencias (ajusta según histogramas de longitud)
MAX_LEN_SRC = 200
MAX_LEN_TGT = 20

BATCH_SIZE = 64
EPOCHS = 15
TEACHER_FORCING_RATIO = 0.5  # Probabilidad de usar teacher forcing durante entrenamiento

# Tokens especiales
PAD_TOKEN = "<pad>"
SOS_TOKEN = "<sos>"
EOS_TOKEN = "<eos>"
UNK_TOKEN = "<unk>"

# -------------------------------------------------
# 1. Funciones de limpieza y tokenización
# -------------------------------------------------
nltk.download('stopwords', quiet=True)
SPANISH_STOP = set(stopwords.words("spanish"))

def limpiar_texto(texto: str) -> str:
    """
    - Minúsculas.
    - Elimina caracteres que no sean letras (incluidas tildes y ñ) ni espacios ni dígitos.
    - Sustituye múltiples espacios por uno solo.
    - Remueve stopwords.
    """
    texto = texto.lower()
    texto = re.sub(r"[^a-záéíóúñü0-9\s]", " ", texto)
    texto = re.sub(r"\s+", " ", texto).strip()
    tokens = texto.split()
    tokens = [t for t in tokens if t not in SPANISH_STOP]
    return " ".join(tokens)

def tokenizar(texto: str) -> list[str]:
    """
    Simplemente divide por espacios (ya que limpiamos antes),
    retornando una lista de tokens.
    """
    return texto.split()

# -------------------------------------------------
# 2. Leer JSON y preparar pares (entrada, salida)
# -------------------------------------------------
def cargar_dataset_json(ruta_json: str):
    """
    Lee un archivo JSON con estructura de lista de objetos:
      [
        {
          "title": "...",
          "category": "...",
          "summit": "...",
          "description": "...",
          "date": "...",
          "autor": "...",
          "tags": "...",
          "url": "..."
        },
        ...
      ]
    Extrae 'summit' como fuente y 'title' como objetivo, los limpia y tokeniza.
    Devuelve dos listas de listas de tokens:
      - documentos_src: lista de listas de tokens tokenizados de 'summit'
      - documentos_tgt: lista de listas de tokens tokenizados de 'title'
    """
    with open(ruta_json, 'r', encoding='utf-8') as f:
        data = json.load(f)

    documentos_src = []
    documentos_tgt = []

    for item in tqdm(data, desc="Leyendo y limpiando JSON"):
        summit = item.get("summit", "")
        title  = item.get("title", "")

        if not summit or not title:
            continue

        summit_limpio = limpiar_texto(str(summit))
        title_limpio  = limpiar_texto(str(title))

        tokens_src = tokenizar(summit_limpio)
        tokens_tgt = tokenizar(title_limpio)

        if len(tokens_src) == 0 or len(tokens_tgt) == 0:
            continue  # descartamos si quedó vacío

        documentos_src.append(tokens_src)
        documentos_tgt.append(tokens_tgt)

    return documentos_src, documentos_tgt

# -------------------------------------------------
# 3. Construir vocabularios (SRC y TGT)
# -------------------------------------------------
from collections import Counter

class Vocab:
    """
    Clase para almacenar el mapeo token->índice e índice->token,
    y convertir listas de tokens a listas de índices (y viceversa).
    """
    def __init__(self, max_size: int):
        self.counter = Counter()
        self.max_size = max_size
        # Diccionarios finales
        self.stoi = {}
        self.itos = {}

    def construir(self, lista_de_textos: list[list[str]]):
        # 1) Contar todos los tokens
        for texto in lista_de_textos:
            self.counter.update(texto)

        # 2) Tomar los N tokens más frecuentes (menos los especiales)
        vocab_tokens = [tok for tok, _ in self.counter.most_common(self.max_size)]
        # 3) Agregar tokens especiales al inicio
        tokens_finales = [PAD_TOKEN, SOS_TOKEN, EOS_TOKEN, UNK_TOKEN] + vocab_tokens

        # 4) Construir mapeos
        for idx, tok in enumerate(tokens_finales):
            self.stoi[tok] = idx
            self.itos[idx] = tok

    def token2idx(self, token: str) -> int:
        return self.stoi.get(token, self.stoi[UNK_TOKEN])

    def idx2token(self, idx: int) -> str:
        return self.itos.get(idx, UNK_TOKEN)

    def __len__(self):
        return len(self.stoi)

# -------------------------------------------------
# 4. Dataset personalizado y DataLoader
# -------------------------------------------------
class TitulosDataset(Dataset):
    """
    Dataset que recibe dos listas de listas de tokens:
      - documentos_src: [[tok1, tok2, ...], [...], ...]
      - documentos_tgt: [[tok1, tok2, ...], [...], ...]
    También recibe los vocabularios construidos (Vocab src y Vocab tgt).
    Devuelve:
      - src_tensor: [longitud_src] con índices
      - tgt_tensor: [longitud_tgt] con índices (incluye <sos> ... <eos>)
    """
    def __init__(self, documentos_src, documentos_tgt, vocab_src: Vocab, vocab_tgt: Vocab):
        assert len(documentos_src) == len(documentos_tgt)
        self.docs_src = documentos_src
        self.docs_tgt = documentos_tgt
        self.vocab_src = vocab_src
        self.vocab_tgt = vocab_tgt

    def __len__(self):
        return len(self.docs_src)

    def __getitem__(self, idx):
        tokens_src = self.docs_src[idx]
        tokens_tgt = self.docs_tgt[idx]

        # Convertir tokens a índices
        src_indices = [ self.vocab_src.token2idx(tok) for tok in tokens_src ]
        # Para el target, agregamos SOS al inicio y EOS al final
        tgt_indices = [ self.vocab_tgt.token2idx(SOS_TOKEN) ] + \
                      [ self.vocab_tgt.token2idx(tok) for tok in tokens_tgt ] + \
                      [ self.vocab_tgt.token2idx(EOS_TOKEN) ]

        return src_indices, tgt_indices

# Los siguientes vocab_src y vocab_tgt se definirán en main,
# pero pad_collate los usará como variables globales:
vocab_src = None
vocab_tgt = None

def pad_collate(batch):
    """
    Función para collate_fn de DataLoader:
    Recibe un batch = lista de tuplas (src_indices, tgt_indices).
    Retorna:
      - src_tensor_padded: (batch_size, MAX_LEN_SRC)
      - tgt_input_padded: (batch_size, MAX_LEN_TGT)  [para decoder_input]
      - tgt_output_padded: (batch_size, MAX_LEN_TGT) [para calcular loss]
    """
    batch_src, batch_tgt = zip(*batch)

    # 1) Para cada secuencia src, recortamos/pad a MAX_LEN_SRC
    src_padded = []
    for seq in batch_src:
        if len(seq) > MAX_LEN_SRC:
            seq = seq[:MAX_LEN_SRC]
        # padding a la derecha con índice de PAD_TOKEN
        pad_len = MAX_LEN_SRC - len(seq)
        seq = seq + [vocab_src.token2idx(PAD_TOKEN)] * pad_len
        src_padded.append(seq)

    # 2) Para cada secuencia tgt, recortamos/pad a MAX_LEN_TGT
    #    Para generar input y output del decoder:
    #    - decoder_input: todo hasta penúltimo token
    #    - decoder_output: todo desde el token 1 (sin <sos>) 
    dec_in_padded = []
    dec_out_padded = []
    for seq in batch_tgt:
        if len(seq) > MAX_LEN_TGT:
            seq = seq[:MAX_LEN_TGT]
        pad_len = MAX_LEN_TGT - len(seq)
        seq = seq + [vocab_tgt.token2idx(PAD_TOKEN)] * pad_len

        # Decoder input es toda la secuencia tal cual (incluye <sos> ... <eos> ... <pad>)
        dec_input = seq
        # Decoder output es el mismo seq desplazado a la izquierda, con PAD al final
        dec_output = seq[1:] + [vocab_tgt.token2idx(PAD_TOKEN)]
        dec_in_padded.append(dec_input)
        dec_out_padded.append(dec_output)

    # Convertir a tensores largos (LongTensor)
    src_tensor = torch.LongTensor(src_padded)
    tgt_input_tensor = torch.LongTensor(dec_in_padded)
    tgt_output_tensor = torch.LongTensor(dec_out_padded)

    return src_tensor.to(DEVICE), tgt_input_tensor.to(DEVICE), tgt_output_tensor.to(DEVICE)

# -------------------------------------------------
# 5. Definir modelo Seq2Seq con atención (Bahdanau)
# -------------------------------------------------
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=vocab_src.token2idx(PAD_TOKEN))
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=False)

    def forward(self, src_seq):
        # src_seq: (batch_size, max_len_src)
        embedded = self.embedding(src_seq)  # (batch, max_len_src, embed_dim)
        outputs, (h, c) = self.lstm(embedded)
        # outputs: (batch, max_len_src, hidden_dim)
        # h, c: (1, batch, hidden_dim)
        return outputs, (h, c)

class BahdanauAttention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        # Para Bahdanau: score = v^T tanh(W1 * dec_hidden + W2 * enc_output)
        self.W1 = nn.Linear(hidden_dim, hidden_dim)
        self.W2 = nn.Linear(hidden_dim, hidden_dim)
        self.V  = nn.Linear(hidden_dim, 1)

    def forward(self, dec_hidden, enc_outputs):
        """
        dec_hidden: (batch, hidden_dim) → estado oculto actual del Decoder (t)
        enc_outputs: (batch, max_len_src, hidden_dim) → salidas del Encoder en todos los timesteps
        Queremos calcular las “atenciones” a cada posición del encoder para este timestep.

        Pasos:
        1) dec_hidden expandido a (batch, max_len_src, hidden_dim)
        2) score = V^T tanh(W1(enc_outputs) + W2(dec_hidden_expandido))
        3) attention_weights = softmax(score, dim=1)  → (batch, max_len_src, 1)
        4) context_vector = suma(attention_weights * enc_outputs, dim=1)  → (batch, hidden_dim)
        """
        batch_size, max_len_src, _ = enc_outputs.size()
        dec_hidden_exp = dec_hidden.unsqueeze(1).repeat(1, max_len_src, 1)  # (batch, max_len_src, hidden_dim)

        energy = torch.tanh(self.W1(enc_outputs) + self.W2(dec_hidden_exp))  # (batch, max_len_src, hidden_dim)
        score = self.V(energy)  # (batch, max_len_src, 1)

        attention_weights = torch.softmax(score, dim=1)  # (batch, max_len_src, 1)
        context_vector = torch.sum(attention_weights * enc_outputs, dim=1)  # (batch, hidden_dim)

        return context_vector, attention_weights  # (batch, hidden_dim), (batch, max_len_src, 1)

class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=vocab_tgt.token2idx(PAD_TOKEN))
        self.lstm = nn.LSTM(embed_dim + hidden_dim, hidden_dim, batch_first=True)
        self.attention = BahdanauAttention(hidden_dim)
        self.fc = nn.Linear(hidden_dim, vocab_size)  # salida sobre vocabulario

    def forward(self, dec_input_token, dec_hidden, dec_cell, enc_outputs):
        """
        dec_input_token: (batch, 1) → índice del token actual a predecir
        dec_hidden: (1, batch, hidden_dim)
        dec_cell:   (1, batch, hidden_dim)
        enc_outputs: (batch, max_len_src, hidden_dim)

        Devuelve:
          - logits: (batch, vocab_size) → logits sin softmax para este timestep
          - new_hidden, new_cell: nuevos estados (1, batch, hidden_dim)
          - attn_weights: (batch, max_len_src, 1)
        """
        embedded = self.embedding(dec_input_token)  # (batch, 1, embed_dim)
        dec_hidden_2d = dec_hidden.squeeze(0)       # (batch, hidden_dim)
        context_vector, attn_weights = self.attention(dec_hidden_2d, enc_outputs)
        embedded_2d = embedded.squeeze(1)           # (batch, embed_dim)

        lstm_input = torch.cat([embedded_2d, context_vector], dim=-1).unsqueeze(1)  # (batch,1,embed+hidden)
        output, (new_hidden, new_cell) = self.lstm(lstm_input, (dec_hidden, dec_cell))
        output = output.squeeze(1)  # (batch, hidden_dim)
        logits = self.fc(output)    # (batch, vocab_size)

        return logits, new_hidden, new_cell, attn_weights

# -------------------------------------------------
# 6. Bucle de entrenamiento
# -------------------------------------------------
def entrenar_epoch(
        encoder: Encoder,
        decoder: Decoder,
        dataloader: DataLoader,
        encoder_optimizer,
        decoder_optimizer,
        criterion
):
    encoder.train()
    decoder.train()

    total_loss = 0

    for src_batch, tgt_in_batch, tgt_out_batch in tqdm(dataloader, desc="Entrenando"):
        batch_size = src_batch.size(0)

        # A) Borra gradientes
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        # B) Forward pass del encoder
        enc_outputs, (h, c) = encoder(src_batch)

        # C) Preparar inputs para el decoder
        dec_hidden = h
        dec_cell   = c
        dec_input_token = torch.LongTensor([vocab_tgt.token2idx(SOS_TOKEN)] * batch_size).unsqueeze(1).to(DEVICE)

        loss = 0

        # D) Iterar sobre cada timestep del target
        for t in range(MAX_LEN_TGT):
            logits, dec_hidden, dec_cell, _ = decoder(
                dec_input_token, dec_hidden, dec_cell, enc_outputs
            )
            ground_truth = tgt_out_batch[:, t]  # (batch,)
            loss_t = criterion(logits, ground_truth)
            loss += loss_t

            use_teacher = True if random.random() < TEACHER_FORCING_RATIO else False
            if use_teacher:
                dec_input_token = tgt_in_batch[:, t].unsqueeze(1)
            else:
                pred_token = logits.argmax(dim=1)
                dec_input_token = pred_token.unsqueeze(1)

        # E) Backpropagation
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item() / MAX_LEN_TGT  # promedio por palabra

    return total_loss / len(dataloader)

# -------------------------------------------------
# 7. Función de inferencia para generar título
# -------------------------------------------------
def generar_titulo(
        texto_src: str,
        encoder: Encoder,
        decoder: Decoder,
        tokenizer_src_vocab: Vocab,
        tokenizer_tgt_vocab: Vocab,
        max_len_src: int,
        max_len_tgt: int
) -> str:
    encoder.eval()
    decoder.eval()

    # 1) Limpiar y tokenizar
    texto_limpio = limpiar_texto(texto_src)
    tokens = tokenizar(texto_limpio)

    # 2) Tokens → índices
    indices = [ tokenizer_src_vocab.token2idx(tok) for tok in tokens ]
    if len(indices) > max_len_src:
        indices = indices[:max_len_src]
    else:
        pad_len = max_len_src - len(indices)
        indices = indices + [ tokenizer_src_vocab.token2idx(PAD_TOKEN) ] * pad_len

    src_tensor = torch.LongTensor(indices).unsqueeze(0).to(DEVICE)  # (1, max_len_src)

    # 3) Encoder forward
    with torch.no_grad():
        enc_outputs, (dec_hidden, dec_cell) = encoder(src_tensor)

    # 4) Inicializar dec_input_token = <sos>
    dec_input_token = torch.LongTensor([ tokenizer_tgt_vocab.token2idx(SOS_TOKEN) ]).unsqueeze(1).to(DEVICE)

    resultado_tokens = []

    # 5) Iterar timesteps en el decoder
    for _ in range(max_len_tgt):
        with torch.no_grad():
            logits, dec_hidden, dec_cell, _ = decoder(dec_input_token, dec_hidden, dec_cell, enc_outputs)
        pred_token_idx = logits.argmax(dim=1).item()  # escalar

        if pred_token_idx == tokenizer_tgt_vocab.token2idx(EOS_TOKEN) or \
                pred_token_idx == tokenizer_tgt_vocab.token2idx(PAD_TOKEN):
            break

        pred_token = tokenizer_tgt_vocab.idx2token(pred_token_idx)
        resultado_tokens.append(pred_token)

        dec_input_token = torch.LongTensor([pred_token_idx]).unsqueeze(1).to(DEVICE)

    return " ".join(resultado_tokens)

### Entrenamiento

In [4]:
# -------------------------------------------------
# 8. Pipeline completo adaptado a JSON
# -------------------------------------------------
# 1) Cargar datos desde JSON
ruta_json = "gestionspider4.json"  # Ajusta el nombre si tu JSON es distinto
docs_src, docs_tgt = cargar_dataset_json(ruta_json)

# 2) Construir vocabularios
print("\nConstruyendo vocabularios...")
vocab_src = Vocab(MAX_VOCAB_SRC)
vocab_src.construir(docs_src)

vocab_tgt = Vocab(MAX_VOCAB_TGT)
vocab_tgt.construir(docs_tgt)

print(f"Tamaño vocab_SRC: {len(vocab_src)}")
print(f"Tamaño vocab_TGT: {len(vocab_tgt)}")

# 3) Crear Dataset y DataLoader
dataset = TitulosDataset(docs_src, docs_tgt, vocab_src, vocab_tgt)
dataloader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=pad_collate
)

# 4) Instanciar modelos
encoder = Encoder(len(vocab_src), EMBEDDING_DIM, HIDDEN_DIM).to(DEVICE)
decoder = Decoder(len(vocab_tgt), EMBEDDING_DIM, HIDDEN_DIM).to(DEVICE)

# 5) Definir optimizadores y función de pérdida
encoder_optimizer = optim.Adam(encoder.parameters(), lr=1e-3)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss(ignore_index=vocab_tgt.token2idx(PAD_TOKEN))

# 6) Entrenamiento
print("\nIniciando entrenamiento:")
for epoch in range(1, EPOCHS + 1):
    loss_epoch = entrenar_epoch(
        encoder, decoder, dataloader,
        encoder_optimizer, decoder_optimizer, criterion
    )
    print(f"Epoch {epoch}/{EPOCHS}  →  Loss promedio: {loss_epoch:.4f}")

Leyendo y limpiando JSON: 100%|██████████| 3761/3761 [00:00<00:00, 54516.66it/s]



Construyendo vocabularios...
Tamaño vocab_SRC: 12241
Tamaño vocab_TGT: 5004

Iniciando entrenamiento:


Entrenando: 100%|██████████| 59/59 [00:36<00:00,  1.61it/s]


Epoch 1/15  →  Loss promedio: nan


Entrenando: 100%|██████████| 59/59 [00:38<00:00,  1.54it/s]


...

Epoch 13/15  →  Loss promedio: nan


Entrenando: 100%|██████████| 59/59 [00:38<00:00,  1.55it/s]


Epoch 14/15  →  Loss promedio: nan


Entrenando: 100%|██████████| 59/59 [00:38<00:00,  1.55it/s]

Epoch 15/15  →  Loss promedio: nan


### Validando Modelo - Generando títulos a las noticias

In [5]:
# 7) Probar inferencia en algunos ejemplos aleatorios del dataset
print("\nGenerando títulos de ejemplo (primeros 5 del dataset):\n")
for i in range(5):
    resumen = " ".join(docs_src[i])  # reconstruyo el texto preprocesado
    titulo_real = " ".join(docs_tgt[i])
    titulo_gen = generar_titulo(
        texto_src=resumen,
        encoder=encoder,
        decoder=decoder,
        tokenizer_src_vocab=vocab_src,
        tokenizer_tgt_vocab=vocab_tgt,
        max_len_src=MAX_LEN_SRC,
        max_len_tgt=MAX_LEN_TGT
    )
    print(f"Detalle (recortado): {' '.join(docs_src[i][:30])}...")
    print(f"  Título real   : {titulo_real}")
    print(f"  Título generado: {titulo_gen}\n")


Generando títulos de ejemplo (primeros 5 del dataset):

Detalle (recortado): autoridad portuaria señala mayoría operadores puertos concesionados interior evalúa acogerse nueva ley permite prorrogar plazos contratos...
  Título real   : apn puerto callao espera us 2 300 millones adicionales extensión concesiones
  Título generado: <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>

Detalle (recortado): tendencia lineal mayo pues inició mes s 3 66 luego repuntó s 3 69...
  Título real   : dólar s 3 62 quiénes convendría comprar
  Título generado: <unk> <unk> <unk> <unk> <unk> <unk> <unk>

Detalle (recortado): municipalidad victoria evalúa nuevo espacio comercial modo sincere ingresos negocios relación alquiler percibido comuna detalles entrevista alcalde rubén cano...
  Título real   : luego tiendas parque cánepa próxima concesión bajo lupa victoria
  Título generado: <unk> <unk> <unk> <unk> <unk> <unk>

Detalle (recortado): asociación gremios productores agrarios perú agap detalló golpe e